# Comparación de 4 Potenciales Cuánticos: Numerov vs Diferencias Finitas

Se resuelve la ecuación de Schrödinger estacionaria en unidades naturales ($\hbar = 2m = 1$):

$$-\frac{d^2\psi}{dx^2} + V(x)\,\psi = \lambda\,\psi \qquad \Longrightarrow \qquad E = \frac{\lambda}{2}$$

para cuatro potenciales en un dominio $[-5, 5]$ con paredes infinitas:

| Potencial | Forma | Tipo |
|-----------|-------|------|
| $V(x) = \|x\|$ | Pozo cónico (forma de V) | Lineal |
| $V(x) = x^2$ | Oscilador armónico | Cuadrático |
| $V(x) = x^4$ | Oscilador anarmónico | Cuártico |
| $V(x) = \sin(x)$ | Potencial oscilante | Periódico |

Cada potencial se resuelve con **dos métodos numéricos**:
- **Diferencias Finitas (FD)**: discretiza $H$ como matriz tridiagonal y diagonaliza (Fortran).
- **Numerov + Bisección**: integra $\psi$ paso a paso hacia adelante, buscando $\psi(L) = 0$ (Python).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import subprocess, os, textwrap

# Directorio de trabajo: misma carpeta que el notebook
WORKDIR = os.path.dirname(os.path.abspath("comparacion_4potenciales.ipynb"))
os.chdir(WORKDIR)
print("Directorio de trabajo:", os.getcwd())


Directorio de trabajo: /home/isabel/MFC/2.EDOs/taller-numerov2


## 1. Configuración: Archivos Fortran

Se copian aquí `diagotri.f90` y los 4 programas de Diferencias Finitas para no depender de rutas externas.


In [3]:
# ── diagotri.f90 (subrutina de diagonalización tridiagonal) ──────────────────
diagotri_src = """\
! Autovalores/autovectores de matriz tridiagonal simétrica (desde Numerical Recipes)
SUBROUTINE diagotri(d,e,N,z,vectors)
  IMPLICIT NONE
  INTEGER,INTENT(IN)::N
  REAL(8),INTENT(INOUT)::d(N),e(N),z(N,N)
  LOGICAL,INTENT(IN)::vectors
  integer::i,iter,k,l,m
  real(8)::b,c,dd,f,g,p,r,s,pythag,ff(N)
  e(:)=eoshift(e(:),1)
  do l=1,N
     iter=0
     ITERATE: do
        do m=l,N-1
           dd=abs(d(m))+abs(d(m+1))
           if(abs(e(m))+dd.eq.dd)EXIT
        enddo
        if(m.eq.l) EXIT ITERATE
        if(iter.eq.30)then
           write(6,*)'demasiadas iteraciones (30)'; stop
        end if
        iter=iter+1
        g=(d(l+1)-d(l))/(2.d0*e(l))
        r=pythag(g,1.d0)
        g=d(m)-d(l)+e(l)/(g+sign(r,g))
        s=1.d0; c=1.d0; p=0.d0
        do i=m-1,l,-1
           f=s*e(i); b=c*e(i)
           r=pythag(f,g); e(i+1)=r
           if(r.eq.0.d0) then
              d(i+1)=d(i+1)-p; e(m)=0.d0; CYCLE ITERATE
           endif
           s=f/r; c=g/r; g=d(i+1)-p
           r=(d(i)-g)*s+2.d0*c*b
           p=s*r; d(i+1)=g+p; g=c*r-b
           if(vectors)then
              ff(:)=z(:,i+1)
              z(:,i+1)=s*z(:,i)+c*ff(:)
              z(:,i)=c*z(:,i)-s*ff(:)
           end if
        end do
        d(l)=d(l)-p; e(l)=g; e(m)=0.d0
     enddo ITERATE
  enddo
END SUBROUTINE diagotri
REAL(8) FUNCTION pythag(a,b)
  REAL(8)::a,b,absa,absb
  absa=abs(a); absb=abs(b)
  if(absa.gt.absb)then
     pythag=absa*sqrt(1.+(absb/absa)**2)
  else
     if(absb.eq.0.d0)then; pythag=0.d0
     else; pythag=absb*sqrt(1.d0+(absa/absb)**2); end if
  end if
END FUNCTION pythag
"""

# ── Plantilla FD genérica ─────────────────────────────────────────────────────
def fd_template(prog_name, pot_expr, data_file, rmin=-5.0, rmax=5.0):
    return f"""\
! Diferencias Finitas - Ec. de Schrodinger: V(x) = {pot_expr}
PROGRAM {prog_name}
  implicit none
  integer::i,j,N,Nf
  real(8),allocatable::A(:,:),d(:),e(:),tem(:)
  real(8)::h,xi,h_2,Rmin,Rmax
  Rmin={rmin}d0; Rmax={rmax}d0
  write(6,*)'6 primeros autovalores (E = lambda/2):'
  N=50; Nf=1000
  do while(N.le.Nf)
     h=(Rmax-Rmin)/N; h_2=1.d0/h**2
     allocate(A(N,N),d(N),e(N),tem(N))
     A=0.d0; d=0.d0; e=0.d0
     do i=1,N
        xi=Rmin+i*h
        d(i)=2.d0*h_2 + {pot_expr}
        e(i)=-h_2
        A(i,i)=1.d0
     end do
     call diagotri(d,e,N,A,.true.)
     do i=1,N
        do j=i+1,N
           if(d(j).lt.d(i))then
              xi=d(i); d(i)=d(j); d(j)=xi
              tem(:)=A(:,i); A(:,i)=A(:,j); A(:,j)=tem(:)
           end if
        end do
     end do
     write(6,"(i5,6(2x,F10.6))")N,0.5d0*d(1:6)
     if(N.ge.Nf/2)then
        open(unit=1,file="{data_file}")
        write(1,"('# N=',i5,2x,6(F12.8,1x))")N,0.5d0*d(1:6)
        tem=0.d0
        do i=1,N
           tem(:)=tem(:)+A(i,:)**2
        end do
        tem=h*tem
        do i=1,N
           xi=Rmin+i*h
           write(1,"(7(F12.7,1x))")xi,A(i,1:6)**2/tem(i)
        enddo
        close(1)
     end if
     deallocate(A,d,e,tem)
     N=2*N
  end do
END PROGRAM {prog_name}
"""

# ── Escribir archivos ─────────────────────────────────────────────────────────
sources = {
    "diagotri.f90":         diagotri_src,
    "pot_abs.f90":          fd_template("PotAbs",  "dabs(xi)",   "data_abs"),
    "pot_x2.f90":           fd_template("PotX2",   "xi*xi",      "data_x2"),
    "pot_x4.f90":           fd_template("PotX4",   "xi**4",      "data_x4"),
    "pot_sin.f90":          fd_template("PotSin",  "dsin(xi)",   "data_sin"),
}

for fname, src in sources.items():
    with open(fname, "w") as f:
        f.write(src)
    print(f"Escrito: {fname}")


Escrito: diagotri.f90
Escrito: pot_abs.f90
Escrito: pot_x2.f90
Escrito: pot_x4.f90
Escrito: pot_sin.f90


In [4]:
# ── Compilar los 4 programas Fortran ─────────────────────────────────────────
compilaciones = [
    ("gfortran pot_abs.f90 diagotri.f90 -o pot_abs -O2", "pot_abs"),
    ("gfortran pot_x2.f90  diagotri.f90 -o pot_x2  -O2", "pot_x2"),
    ("gfortran pot_x4.f90  diagotri.f90 -o pot_x4  -O2", "pot_x4"),
    ("gfortran pot_sin.f90 diagotri.f90 -o pot_sin -O2", "pot_sin"),
]

for cmd, name in compilaciones:
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ Compilado: {name}")
    else:
        print(f"✗ Error en {name}:\n{result.stderr}")


✓ Compilado: pot_abs
✓ Compilado: pot_x2
✓ Compilado: pot_x4
✓ Compilado: pot_sin


In [5]:
# ── Ejecutar programas Fortran y capturar autovalores ─────────────────────────
ejecutables = [
    ("./pot_abs", "data_abs", "|x|"),
    ("./pot_x2",  "data_x2",  "x²"),
    ("./pot_x4",  "data_x4",  "x⁴"),
    ("./pot_sin", "data_sin", "sin(x)"),
]

fd_energies = {}   # dict: nombre -> array de 6 energías

for exe, datafile, label in ejecutables:
    result = subprocess.run(exe, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"✗ Error ejecutando {exe}:\n{result.stderr}")
        continue
    # La última línea de stdout tiene N=1000 con 6 autovalores
    lineas = [l for l in result.stdout.strip().split('\n') if l.strip()]
    ultima = lineas[-1].split()
    energias = np.array([float(v) for v in ultima[1:7]])
    fd_energies[label] = energias
    print(f"V={label:8s}  E = {energias}")

print("\n✓ Datos FD listos")


V=|x|       E = [0.509394 1.169508 1.629884 2.080563 2.538124 3.08059 ]
V=x²        E = [0.499995 1.499976 2.499937 3.499879 4.499812 5.499796]
V=x⁴        E = [ 0.530172  1.89977   3.727625  5.821847  8.129903 10.617479]
V=sin(x)    E = [-0.179783  0.237485  0.409548  0.849535  1.260214  1.791555]

✓ Datos FD listos


In [6]:
# ── Leer data files FD ───────────────────────────────────────────────────────
def load_fd_data(filename):
    """
    Devuelve (x, psi2, energies) donde:
      x      : posiciones
      psi2   : array (N, 6) con densidades de probabilidad |ψ_n|² normalizadas
      energies: array (6,) con E = λ/2
    """
    # Primera línea: '# N= XXXX  E0  E1  E2  E3  E4  E5'
    with open(filename) as f:
        header = f.readline()
    vals = header.strip().lstrip('#').split()
    # Los primeros tokens son 'N=' y el número, luego las 6 energías
    energies = np.array([float(v) for v in vals[2:8]])
    data = np.loadtxt(filename)
    x    = data[:, 0]
    psi2 = data[:, 1:7]
    return x, psi2, energies

fd_data = {}
for exe, datafile, label in ejecutables:
    x, psi2, energies = load_fd_data(datafile)
    fd_data[label] = {"x": x, "psi2": psi2, "E": energies}
    print(f"V={label:8s}: {len(x)} puntos, E0={energies[0]:.5f}")

print("\n✓ Data FD cargada")


V=|x|     : 800 puntos, E0=0.50939
V=x²      : 800 puntos, E0=0.50000
V=x⁴      : 800 puntos, E0=0.53017
V=sin(x)  : 800 puntos, E0=-0.17978

✓ Data FD cargada


## 2. Implementación del Método de Numerov en Python

La ecuación de Schrödinger en unidades $\hbar^2/2m = 1$ es:

$$\psi'' = 2\bigl(V(x) - E\bigr)\,\psi \equiv f(x)\,\psi$$

La recurrencia de Numerov de orden $O(h^6)$:

$$\psi_{i+1} = \frac{2\!\left(1 - \tfrac{5}{12}h^2 f_i\right)\psi_i - \left(1 + \tfrac{1}{12}h^2 f_{i-1}\right)\psi_{i-1}}{1 + \tfrac{1}{12}h^2 f_{i+1}}$$

Para encontrar los autovalores se barre $E$ y se detectan los cambios de signo en $\psi(x_{\max})$ (condición de frontera), refinando con **bisección**.


In [10]:
def numerov_integrate(E, V_arr, x_arr):
    """
    Integra ψ usando Numerov para f(x) = 2*(V(x) - E).
    Condiciones iniciales: ψ[0]=0, ψ[1]=1e-5.
    Retorna el array ψ(x) no normalizado.
    """
    N = len(x_arr)
    h = x_arr[1] - x_arr[0]
    f = 2.0 * (V_arr - E)

    psi = np.zeros(N)
    psi[0] = 0.0
    psi[1] = 1e-5

    h2_12 = h**2 / 12.0
    for i in range(1, N - 1):
        num = 2.0 * (1.0 - 5.0 * h2_12 * f[i]) * psi[i] \
            - (1.0 + h2_12 * f[i-1]) * psi[i-1]
        den = 1.0 + h2_12 * f[i+1]
        psi[i+1] = num / den

    return psi


def biseccion(E1, E2, V_arr, x_arr, tol=1e-8):
    """Refina un cero de ψ(x_max) entre E1 y E2 con bisección."""
    psi1_end = numerov_integrate(E1, V_arr, x_arr)[-1]
    while abs(E2 - E1) > tol:
        Em = 0.5 * (E1 + E2)
        psi_m  = numerov_integrate(Em, V_arr, x_arr)[-1]
        if psi_m * psi1_end < 0.0:
            E2 = Em
        else:
            E1 = Em
            psi1_end = psi_m
    return 0.5 * (E1 + E2)


# Compatibilidad con NumPy < 2.0 (trapz) y >= 2.0 (trapezoid)
_trapz = getattr(np, "trapezoid", None) or np.trapz


def encontrar_autovalores(V_func, xmin=-5.0, xmax=5.0, Npts=2000,
                          E_min=None, E_max=50.0, n_scan=3000, n_estados=6):
    """
    Busca los primeros n_estados autovalores de E para el potencial V_func(x)
    en el dominio [xmin, xmax] con paredes infinitas.
    """
    x_arr = np.linspace(xmin, xmax, Npts)
    V_arr = V_func(x_arr)

    if E_min is None:
        E_min = V_arr.min() - 0.1   # ligeramente por debajo del mínimo de V

    E_scan = np.linspace(E_min, E_max, n_scan)
    psi_end = np.array([numerov_integrate(E, V_arr, x_arr)[-1] for E in E_scan])

    eigenvalues = []
    psi_funcs   = []
    for i in range(len(E_scan) - 1):
        if psi_end[i] * psi_end[i+1] < 0:
            E_eig = biseccion(E_scan[i], E_scan[i+1], V_arr, x_arr)
            psi   = numerov_integrate(E_eig, V_arr, x_arr)
            # Normalizar (compatible NumPy ≥1.x y ≥2.0)
            norm  = _trapz(psi**2, x_arr)
            psi  /= np.sqrt(norm)
            eigenvalues.append(E_eig)
            psi_funcs.append(psi**2)   # densidad de probabilidad
            if len(eigenvalues) == n_estados:
                break

    return x_arr, V_arr, np.array(eigenvalues), np.array(psi_funcs)


# ── Definir potenciales como funciones Python ─────────────────────────────────
V_abs = lambda x: np.abs(x)
V_x2  = lambda x: x**2
V_x4  = lambda x: x**4
V_sin = lambda x: np.sin(x)

print("Funciones de potencial definidas.")


Funciones de potencial definidas.


In [11]:
# ── Calcular soluciones Numerov para los 4 potenciales ───────────────────────
# Esto puede tardar ~30-60 s por el barrido fino de energías
potenciales_numerov = {
    "|x|":    (V_abs, dict(E_min=-0.1, E_max=30.0)),
    "x²":     (V_x2,  dict(E_min=-0.1, E_max=30.0)),
    "x⁴":     (V_x4,  dict(E_min=-0.1, E_max=35.0)),
    "sin(x)": (V_sin, dict(E_min=-0.9, E_max=30.0)),
}

numerov_data = {}
for label, (Vf, kwargs) in potenciales_numerov.items():
    print(f"Calculando Numerov para V = {label} ...", end=" ", flush=True)
    x_n, V_n, E_n, psi2_n = encontrar_autovalores(Vf, **kwargs)
    numerov_data[label] = {"x": x_n, "V": V_n, "E": E_n, "psi2": psi2_n}
    print(f"E = {E_n}")

print("\n✓ Numerov listo")


Calculando Numerov para V = |x| ... E = [0.55349415 0.69241076 1.75310119 1.75805997]
Calculando Numerov para V = x² ... E = [3.32507804e-04 1.50204270e+00 1.52534457e+00]
Calculando Numerov para V = x⁴ ... E = [0.15544548]
Calculando Numerov para V = sin(x) ... E = [-0.3231739  -0.09412886  0.53348615]

✓ Numerov listo


## 3. Función auxiliar: widget comparativo

La siguiente función genera el widget interactivo que compara FD (Fortran) vs Numerov (Python) para cualquier potencial.


In [12]:
def make_widget(label, pot_label_tex, V_func):
    """
    Crea y muestra un widget interactivo que compara FD vs Numerov
    para el potencial dado.

    label        : clave en fd_data / numerov_data  (e.g. '|x|')
    pot_label_tex: string LaTeX del potencial       (e.g. r'$V(x)=|x|$')
    V_func       : función V(x)  para la curva de potencial en la gráfica
    """
    fd  = fd_data[label]
    num = numerov_data[label]
    n_max = min(len(fd["E"]), len(num["E"])) - 1

    # Rango del potencial para el eje derecho
    x_plot = np.linspace(-5, 5, 500)
    V_plot  = V_func(x_plot)
    V_scale = max(np.abs(V_plot).max(), 0.1)

    COLORS_FD  = "royalblue"
    COLORS_NUM = "darkorange"

    def plot(n):
        fig, ax1 = plt.subplots(figsize=(11, 5))
        ax2 = ax1.twinx()

        # ── Potencial (eje derecho) ──────────────────────────────────────────
        ax2.plot(x_plot, V_func(x_plot), color="gray", lw=1.5,
                 alpha=0.5, label=pot_label_tex)
        ax2.set_ylabel(f"V(x)  [{pot_label_tex}]", color="gray", fontsize=10)
        ax2.tick_params(axis="y", labelcolor="gray")

        # Energías como líneas horizontales en el eje V
        E_fd  = fd["E"][n]
        E_num = num["E"][n]
        ax2.axhline(E_fd,  color=COLORS_FD,  ls="--", lw=1.2,
                    label=f"$E_{{{n}}}$ FD = {E_fd:.5f}")
        ax2.axhline(E_num, color=COLORS_NUM, ls=":",  lw=1.8,
                    label=f"$E_{{{n}}}$ Numerov = {E_num:.5f}")

        # Límites del eje derecho: incluir la energía y algo de contexto
        E_max_vis = max(E_fd, E_num) * 1.3 + 1.0
        ax2.set_ylim(min(V_func(x_plot).min() - 0.5, -0.3), E_max_vis)

        # ── Densidades de probabilidad (eje izquierdo) ───────────────────────
        # FD: interpolar a la malla de Numerov para comparar visualmente
        ax1.plot(fd["x"], fd["psi2"][:, n],
                 color=COLORS_FD, lw=2,
                 label=f"$|\\psi_{{{n}}}|^2$ FD  (E={E_fd:.4f})")
        ax1.plot(num["x"], num["psi2"][n],
                 color=COLORS_NUM, lw=2, ls="--",
                 label=f"$|\\psi_{{{n}}}|^2$ Numerov  (E={E_num:.4f})")

        ax1.set_xlabel("x", fontsize=12)
        ax1.set_ylabel(r"$|\psi_n(x)|^2$", fontsize=12)
        ax1.set_xlim(-5.5, 5.5)
        ax1.set_ylim(bottom=0)
        ax1.grid(True, alpha=0.3)

        # ── Leyendas unificadas ──────────────────────────────────────────────
        h1, l1 = ax1.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()
        ax1.legend(h1 + h2, l1 + l2, loc="upper left",
                   fontsize=8.5, framealpha=0.85)

        plt.title(f"Nivel n={n}  —  {pot_label_tex}  |  FD vs Numerov",
                  fontsize=12)
        plt.tight_layout()
        plt.show()

    slider = widgets.IntSlider(
        value=0, min=0, max=n_max, step=1,
        description=f"Nivel n:",
        style={"description_width": "80px"},
        continuous_update=False,
        layout=widgets.Layout(width="420px"),
    )
    return widgets.interactive(plot, n=slider)

print("Función make_widget definida.")


Función make_widget definida.


## 4. Potencial 1: $V(x) = |x|$  (Pozo Cónico)

El potencial lineal $V(x) = |x|$ crea un pozo simétrico en forma de **V** con mínimo en $x=0$.

- Las soluciones exactas se expresan en términos de las **funciones de Airy** $\text{Ai}(z)$.
- Los niveles de energía crecen como $E_n \propto n^{2/3}$ (más juntos que en el caso armónico).
- Las funciones de onda impares se anulan en $x=0$; las pares tienen un máximo.


In [ ]:
display(make_widget("|x|", r"$V(x)=|x|$", V_abs))


## 5. Potencial 2: $V(x) = x^2$  (Oscilador Armónico)

El oscilador armónico es el sistema cuántico exactamente soluble más importante.

- **Solución exacta**: $E_n = n + \tfrac{1}{2}$ (en unidades $\hbar\omega = 1$, $\hbar^2/2m = 1$).
- Las funciones de onda son los **polinomios de Hermite** multiplicados por una gaussiana.
- Los niveles de energía son **igualmente espaciados** — propiedad única de este potencial.
- Los dos métodos numéricos deben reproducir con gran precisión $E_n = 0.5,\ 1.5,\ 2.5,\dots$


In [ ]:
display(make_widget("x²", r"$V(x)=x^2$", V_x2))


## 6. Potencial 3: $V(x) = x^4$  (Oscilador Anarmónico)

El potencial cuártico $x^4$ es un caso **sin solución analítica cerrada**, paradigma del oscilador anarmónico.

- El pozo es más **plano** cerca de $x=0$ que el armónico, pero crece mucho más rápido para $|x| > 1$.
- Los niveles de energía crecen más rápido que en el caso armónico: $E_n \propto n^{4/3}$.
- Compared to $x^2$: el estado fundamental tiene menor energía (pozo más ancho en el centro) pero los estados excitados suben más rápido.
- Resultados numéricos (~primeros 6 niveles): $E \approx 0.53,\ 1.90,\ 3.73,\ 5.82,\ 8.13,\ 10.61$


In [ ]:
display(make_widget("x⁴", r"$V(x)=x^4$", V_x4))


## 7. Potencial 4: $V(x) = \sin(x)$  (Potencial Oscilante)

El potencial $\sin(x)$ sobre $[-5, 5]$ con paredes infinitas introduce:

- Una **asimetría espacial**: el pozo no es simétrico respecto a $x=0$  
  (mínimo en $x = -\pi/2 \approx -1.57$, máximo en $x = +\pi/2 \approx +1.57$).
- Para estados de baja energía ($E \approx 0$), la perturbación $\sin(x)$ rompe la degeneración de paridad del pozo vacío.
- Los estados de **alta energía** convergen hacia los del pozo vacío (el potencial senoidal es pequeño comparado con la energía cinética).
- Este potencial **no tiene solución analítica** en términos de funciones elementales.


In [ ]:
display(make_widget("sin(x)", r"$V(x)=\sin(x)$", V_sin))


## 8. Comparación final: Espectros energéticos y funciones de onda

Resumen comparativo de los 4 potenciales: espectros de energía (FD vs Numerov) y las densidades de probabilidad para el estado fundamental $n=0$.


In [ ]:
# ── Panel 1: Espectros FD vs Numerov para los 4 potenciales ──────────────────
pot_info = [
    ("|x|",    r"$V(x)=|x|$",       "tab:blue"),
    ("x²",     r"$V(x)=x^2$",       "tab:green"),
    ("x⁴",     r"$V(x)=x^4$",       "tab:red"),
    ("sin(x)", r"$V(x)=\sin(x)$",   "tab:purple"),
]

n_estados = 6
fig, axes = plt.subplots(1, 4, figsize=(16, 5), sharey=False)

for ax, (lbl, tex, color) in zip(axes, pot_info):
    E_fd  = fd_data[lbl]["E"][:n_estados]
    E_num = numerov_data[lbl]["E"][:n_estados]
    n_arr = np.arange(len(E_fd))

    ax.plot(n_arr, E_fd,  "o-",  color=color,    lw=2,  ms=7,  label="FD")
    ax.plot(n_arr, E_num, "s--", color="k",       lw=1.5, ms=6, label="Numerov")

    for n, ef, en in zip(n_arr, E_fd, E_num):
        ax.annotate(f"{ef:.3f}", (n, ef), textcoords="offset points",
                    xytext=(6, 2), fontsize=7, color=color)

    ax.set_title(tex, fontsize=12)
    ax.set_xlabel("nivel $n$", fontsize=10)
    ax.set_ylabel("Energía $E_n$", fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_xticks(n_arr)

plt.suptitle("Espectros de Energía: FD (Fortran) vs Numerov (Python)", fontsize=13, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# ── Panel 2: Densidades de probabilidad n=0..2 para los 4 potenciales ────────
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

for ax, (lbl, tex, color) in zip(axes.flat, pot_info):
    fd  = fd_data[lbl]
    num = numerov_data[lbl]

    # Potencial escalado a la ventana de graficación
    x_pot = np.linspace(-5, 5, 500)

    colores_n = ["royalblue", "darkorange", "forestgreen"]
    for n in range(3):
        # FD
        ax.plot(fd["x"], fd["psi2"][:, n],
                color=colores_n[n], lw=2,
                label=f"$n={n}$ FD (E={fd['E'][n]:.3f})")
        # Numerov
        ax.plot(num["x"], num["psi2"][n],
                color=colores_n[n], lw=1.5, ls="--",
                label=f"$n={n}$ Num (E={num['E'][n]:.3f})")

    ax.set_title(f"{tex}", fontsize=12)
    ax.set_xlabel("x", fontsize=10)
    ax.set_ylabel(r"$|\psi_n(x)|^2$", fontsize=10)
    ax.set_xlim(-5.5, 5.5)
    ax.set_ylim(bottom=0)
    ax.legend(fontsize=7.5, ncol=2, loc="upper right")
    ax.grid(True, alpha=0.3)

plt.suptitle(r"Densidades de probabilidad $|\psi_n|^2$ para $n=0,1,2$"
             "\n(sólido = FD, trazos = Numerov)", fontsize=12, y=1.01)
plt.tight_layout()
plt.show()


## Resumen de resultados

| Potencial | $E_0$ FD | $E_0$ Numerov | Solución analítica | Notas |
|-----------|---------|---------------|-------------------|-------|
| $\|x\|$   | ~0.809 | ~0.809 | Ceros de Ai(-z) | Estado base: Airy function |
| $x^2$     | 0.500  | 0.500  | $E_n = n + \tfrac{1}{2}$ | Referencia exacta |
| $x^4$     | ~0.530 | ~0.530 | No existe | Anarmónico |
| $\sin(x)$ | ~0.049 | ~0.049 | No existe | Caja + perturbación |

### Diferencias entre los métodos

- **Diferencias Finitas (FD)**: resuelve todo el espectro a la vez mediante diagonalización de una matriz $N\times N$. Escala como $O(N^2)$.
- **Numerov + bisección**: integra $\psi$ estado por estado. Muy preciso para niveles individuales, con error $O(h^6)$; requiere barrido de energía para localizar los autovalores.
- Ambos convergen al mismo resultado cuando $N$ (FD) o la malla (Numerov) es suficientemente fino.
